In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [4]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5

START_DATE = "20240101"
END_DATE = "20241231"

# Build base name
RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATUREs.pkl"

In [9]:
DIRECTION_LABEL_FILE_PATH = '../data/processed/USDJPY-5m-20240101-20241231_FEATURES.pkl'

In [10]:
from torch.utils.data import Dataset, DataLoader

In [11]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
labels = pd.read_pickle(DIRECTION_LABEL_FILE_PATH)

In [12]:
print(df.shape)
df.head()

(73782, 61)


,timestamp,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
929,2024-01-05 03:25:00,144.8960,144.9415,144.8940,144.9105,2.780660e+12,0.006927,0.000100,0.0145,0.000100,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
930,2024-01-05 03:30:00,144.9100,144.9490,144.9060,144.9470,2.220040e+12,0.006398,0.000252,0.0365,0.000252,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
931,2024-01-05 03:35:00,144.9465,144.9490,144.9230,144.9240,3.450710e+12,0.006930,-0.000159,-0.0230,-0.000159,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
932,2024-01-05 03:40:00,144.9245,144.9460,144.9155,144.9430,2.273510e+12,0.007072,0.000131,0.0190,0.000131,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025
933,2024-01-05 03:45:00,144.9435,144.9510,144.9355,144.9465,1.925300e+12,0.006585,0.000024,0.0035,0.000024,...,5,1,0.707107,0.707107,-0.433884,-0.900969,0.848644,0.528964,0.5,0.866025


In [15]:
print(labels.shape)
labels.head()

(3157, 5)


,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2024-02-16 18:45:00,2024-02-16 19:45:00,0.000585,-0.000751,-1.0,0.0
2024-02-16 19:00:00,2024-02-16 19:45:00,0.000581,-0.000793,-1.0,0.0
2024-02-16 19:15:00,2024-02-16 19:45:00,0.000566,-0.000714,-1.0,0.0
2024-02-16 19:30:00,2024-02-16 20:00:00,0.000550,-0.000603,-1.0,0.0
2024-02-18 22:00:00,2024-02-18 23:15:00,0.000451,0.000469,1.0,2.0
